# Model Development

### Prepping data for analysis

In [259]:
!pip3 install tensorflow scikit-learn pandas matplotlib seaborn numpy 

In [260]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [261]:
# Read all the CSV files in the directory
data_dir = '../data'
all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
df_list = [pd.read_csv(f) for f in all_files]
data = pd.concat(df_list, ignore_index=True)

# Sort by datatimestamp
data = data.sort_values(by='Date').reset_index(drop=True)

# Display the first few rows of the combined dataframe
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,year_month
0,2004-01-04,0:00:00,1.6,1143.0,106.0,6.3,825.0,96.0,986.0,86.0,1477.0,978.0,12.0,61.6,0.8593,2004-01
1,2004-01-04,22:00:00,2.0,1186.0,188.0,9.9,976.0,127.0,844.0,113.0,1570.0,1200.0,15.8,48.0,0.8569,2004-01
2,2004-01-04,21:00:00,2.5,1192.0,254.0,10.8,1007.0,154.0,827.0,124.0,1604.0,1223.0,16.8,44.0,0.8341,2004-01
3,2004-01-04,20:00:00,4.9,1536.0,655.0,20.2,1302.0,269.0,666.0,161.0,1922.0,1599.0,18.3,40.9,0.8493,2004-01
4,2004-01-04,19:00:00,5.5,1592.0,840.0,25.0,1429.0,267.0,624.0,164.0,2089.0,1644.0,20.8,36.7,0.8878,2004-01


In [262]:
data.isna().sum()

Date                0
Time                0
CO(GT)           1283
PT08.S1(CO)       385
NMHC(GT)         6784
C6H6(GT)          385
PT08.S2(NMHC)     385
NOx(GT)          1495
PT08.S3(NOx)      385
NO2(GT)          1499
PT08.S4(NO2)      385
PT08.S5(O3)       385
T                 385
RH                385
AH                385
year_month          0
dtype: int64

### Feature Engineering

In [263]:
# Creating Date related columns from 'Date' column
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['IsWeekend'] = data['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Convert time column to datetime
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S')
data['Hour'] = data['Time'].dt.hour

data = data.drop(columns=['Time', 'year_month', 'Date'])

data.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,DayOfWeek,IsWeekend,Hour
0,1.6,1143.0,106.0,6.3,825.0,96.0,986.0,86.0,1477.0,978.0,12.0,61.6,0.8593,2004,1,4,6,1,0
1,2.0,1186.0,188.0,9.9,976.0,127.0,844.0,113.0,1570.0,1200.0,15.8,48.0,0.8569,2004,1,4,6,1,22
2,2.5,1192.0,254.0,10.8,1007.0,154.0,827.0,124.0,1604.0,1223.0,16.8,44.0,0.8341,2004,1,4,6,1,21
3,4.9,1536.0,655.0,20.2,1302.0,269.0,666.0,161.0,1922.0,1599.0,18.3,40.9,0.8493,2004,1,4,6,1,20
4,5.5,1592.0,840.0,25.0,1429.0,267.0,624.0,164.0,2089.0,1644.0,20.8,36.7,0.8878,2004,1,4,6,1,19


In [264]:
# Creating Season column
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

data['Season'] = data['Month'].apply(get_season)

# One-hot encoding for Season
data = pd.get_dummies(data, columns=['Season'], drop_first=True)

In [265]:
# One hot encoding for Year
data = pd.get_dummies(data, columns=['Year'], drop_first=True)

In [266]:
# Cyclical Encoding for hour
data['Hour_sin'] = np.sin(2 * np.pi * data['Hour'] / 24)
data['Hour_cos'] = np.cos(2 * np.pi * data['Hour'] / 24)
data = data.drop(columns=['Hour'])

In [267]:
# Cycilical Encoding for Month
data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)
data = data.drop(columns=['Month'])

In [268]:
# Handle NaN values
data.fillna(data.mean(), inplace=True)

In [269]:
data.columns

Index(['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH', 'Day', 'DayOfWeek', 'IsWeekend', 'Season_spring',
       'Season_summer', 'Season_winter', 'Year_2005', 'Hour_sin', 'Hour_cos',
       'Month_sin', 'Month_cos'],
      dtype='object')

### Baseline Prediction

In [270]:
# Naive pred using prev value
targets = ['CO(GT)', 'C6H6(GT)', 'NMHC(GT)', 'NOx(GT)', 'NO2(GT)']  # add others as needed
baseline_results = {}

for target in targets:
    y = data[target].fillna(method='ffill')  
    y_pred = y.shift(1).iloc[1:]            
    y_true = y.iloc[1:]

    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    baseline_results[target] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
baseline_results_df = pd.DataFrame(baseline_results).T
baseline_results_df

/var/folders/h6/93p_2snd3bl3v008snqlzq7m0000gn/T/ipykernel_71338/1916225933.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = data[target].fillna(method='ffill')
/var/folders/h6/93p_2snd3bl3v008snqlzq7m0000gn/T/ipykernel_71338/1916225933.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = data[target].fillna(method='ffill')
/var/folders/h6/93p_2snd3bl3v008snqlzq7m0000gn/T/ipykernel_71338/1916225933.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = data[target].fillna(method='ffill')
/var/folders/h6/93p_2snd3bl3v008snqlzq7m0000gn/T/ipykernel_71338/1916225933.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = data[tar

,MAE,RMSE,R2
CO(GT),0.684330,1.141277,0.243215
C6H6(GT),3.736177,33.390092,0.313139
NMHC(GT),12.280400,3126.714984,0.324911
NOx(GT),78.551155,17431.584722,0.455097
NO2(GT),21.128452,1030.288784,0.446477


### Define Features and Target Variable

In [271]:
# Define features and target variable (CO)
targetCO = data['CO(GT)']
otherSensorsCO = ['PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)']
featuresCO = data[['PT08.S1(CO)', 'T', 'RH', 'AH', 'Year_2005', 'Day', 'DayOfWeek', 'IsWeekend', 'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos', 'Season_spring', 'Season_summer'] + otherSensorsCO]

In [272]:
# Define features and target variable (NOx)
targetNOx = data['NOx(GT)']
otherSensorsNOx = ['PT08.S2(NMHC)', 'PT08.S1(CO)', 'PT08.S4(NO2)', 'PT08.S5(O3)']
featuresNOx = data[['PT08.S3(NOx)', 'T', 'RH', 'AH', 'Year_2005', 'Day', 'DayOfWeek', 'IsWeekend', 'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos', 'Season_spring', 'Season_summer'] + otherSensorsNOx]

In [273]:
# Define features and target variable (NMHC)
targetNMHC = data['NMHC(GT)']
otherSensorsNMHC = ['PT08.S1(CO)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)']
featuresNMHC = data[['PT08.S2(NMHC)', 'T', 'RH', 'AH', 'Year_2005', 'Day', 'DayOfWeek', 'IsWeekend', 'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos', 'Season_spring', 'Season_summer'] + otherSensorsNMHC]

In [274]:
# Define features and target variable (NO2)
targetNO2 = data['NO2(GT)']
otherSensorsNO2 = ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S5(O3)']
featuresNO2 = data[['PT08.S4(NO2)', 'T', 'RH', 'AH', 'Year_2005', 'Day', 'DayOfWeek', 'IsWeekend', 'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos', 'Season_spring', 'Season_summer'] + otherSensorsNO2]

In [275]:
# Define features and target variable (C6H6)
targetC6H6 = data['C6H6(GT)']
otherSensorsC6H6 = ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)']
featuresC6H6 = data[['PT08.S5(O3)', 'T', 'RH', 'AH', 'Year_2005', 'Day', 'DayOfWeek', 'IsWeekend', 'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos', 'Season_spring', 'Season_summer'] + otherSensorsC6H6]

### Random Forest for Forecasting

In [276]:
# Random forest for CO prediction
# Split the data into train and test sets
# Using a time-based split to respect temporal order
split_idx = int(len(featuresCO) * 0.8)
X_train, X_test = featuresCO.iloc[:split_idx], featuresCO.iloc[split_idx:]
y_train, y_test = targetCO.iloc[:split_idx], targetCO.iloc[split_idx:]

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
y_predCO = rf.predict(X_test)

# Evaluate the model
mseCO = mean_squared_error(y_test, y_predCO)
r2CO = r2_score(y_test, y_predCO)
maeCO = np.mean(np.abs(y_test - y_predCO))

print(f"Random Forest MSE: {mseCO:.4f}")
print(f"Random Forest R2: {r2CO:.4f}")
print(f"Random Forest MAE: {maeCO:.4f}")

Random Forest MSE: 0.6473
Random Forest R2: 0.6632
Random Forest MAE: 0.5388


In [277]:
print(y_train.describe())

count    6132.000000
mean        2.088900
std         1.182016
min         0.100000
25%         1.300000
50%         2.127135
75%         2.500000
max         9.400000
Name: CO(GT), dtype: float64


In [278]:
# Random forest for NOx prediction
# Split the data into train and test sets
# Using a time-based split to respect temporal order
split_idx = int(len(featuresNOx) * 0.8)
X_train, X_test = featuresNOx.iloc[:split_idx], featuresNOx.iloc[split_idx:]
y_train, y_test = targetNOx.iloc[:split_idx], targetNOx.iloc[split_idx:]

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
y_predNOx = rf.predict(X_test)

# Evaluate the model
mseNOx = mean_squared_error(y_test, y_predNOx)
r2NOx = r2_score(y_test, y_predNOx)
maeNOx = np.mean(np.abs(y_test - y_predNOx))

print(f"Random Forest NOx MSE: {mseNOx:.4f}")
print(f"Random Forest NOx R2: {r2NOx:.4f}")
print(f"Random Forest NOx MAE: {maeNOx:.4f}")

Random Forest NOx MSE: 20396.1630
Random Forest NOx R2: 0.5779
Random Forest NOx MAE: 96.4616


In [279]:
print(y_train.describe())

count    6132.000000
mean      209.267368
std       153.067881
min         2.000000
25%        98.000000
50%       203.500000
75%       239.202722
max      1247.000000
Name: NOx(GT), dtype: float64


In [280]:
# Random forest for NMHC prediction
# Split the data into train and test sets
# Using a time-based split to respect temporal order
split_idx = int(len(featuresNMHC) * 0.8)
X_train, X_test = featuresNMHC.iloc[:split_idx], featuresNMHC.iloc[split_idx:]
y_train, y_test = targetNMHC.iloc[:split_idx], targetNMHC.iloc[split_idx:]

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
y_predNMHC = rf.predict(X_test)

# Evaluate the model
mseNMHC = mean_squared_error(y_test, y_predNMHC)
r2NMHC = r2_score(y_test, y_predNMHC)
maeNMHC = np.mean(np.abs(y_test - y_predNMHC))

print(f"Random Forest NMHC MSE: {mseNMHC:.4f}")
print(f"Random Forest NMHC R2: {r2NMHC:.4f}")
print(f"Random Forest NMHC MAE: {maeNMHC:.4f}")

Random Forest NMHC MSE: 3988.0399
Random Forest NMHC R2: -4936954284135370029490853904384.0000
Random Forest NMHC MAE: 41.5488


In [281]:
print(y_train.describe())

count    6132.000000
mean      205.751701
std        76.105317
min         7.000000
25%       205.751701
50%       205.751701
75%       205.751701
max      1084.000000
Name: NMHC(GT), dtype: float64


In [282]:
# Random forest for NO2 prediction
# Split the data into train and test sets
# Using a time-based split to respect temporal order
split_index = int(0.8 * len(data))
X_train, X_test = featuresNO2.iloc[:split_index], featuresNO2.iloc[split_index:]
y_train, y_test = targetNO2.iloc[:split_index], targetNO2.iloc[split_index:]

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
y_predNO2 = rf.predict(X_test)

# Evaluate the model
mseNO2 = mean_squared_error(y_test, y_predNO2)
r2NO2 = r2_score(y_test, y_predNO2)
maeNO2 = np.mean(np.abs(y_test - y_predNO2))

print(f"Random Forest NO2 MSE: {mseNO2:.4f}")
print(f"Random Forest NO2 R2: {r2NO2:.4f}")
print(f"Random Forest NO2 MAE: {maeNO2:.4f}")

Random Forest NO2 MSE: 1670.3740
Random Forest NO2 R2: 0.3507
Random Forest NO2 MAE: 29.5535


In [283]:
print(y_train.describe())

count    6132.000000
mean      100.014217
std        33.850669
min         5.000000
25%        76.000000
50%       110.384952
75%       114.000000
max       233.000000
Name: NO2(GT), dtype: float64


In [284]:
# Random forest for C6H6 prediction
# Split the data into train and test sets
# Using a time-based split to respect temporal order
split_idx = int(len(featuresC6H6) * 0.8)
X_train, X_test = featuresC6H6.iloc[:split_idx], featuresC6H6.iloc[split_idx:]
y_train, y_test = targetC6H6.iloc[:split_idx], targetC6H6.iloc[split_idx:]

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
y_predC6H6 = rf.predict(X_test)

# Evaluate the model
mseC6H6 = mean_squared_error(y_test, y_predC6H6)
r2C6H6 = r2_score(y_test, y_predC6H6)
maeC6H6 = np.mean(np.abs(y_test - y_predC6H6))

print(f"Random Forest C6H6 MSE: {mseC6H6:.4f}")
print(f"Random Forest C6H6 R2: {r2C6H6:.4f}")
print(f"Random Forest C6H6 MAE: {maeC6H6:.4f}")

Random Forest C6H6 MSE: 0.0009
Random Forest C6H6 R2: 1.0000
Random Forest C6H6 MAE: 0.0129


In [285]:
print(y_train.describe())

count    6132.000000
mean       10.322155
std         7.174877
min         0.200000
25%         5.000000
50%         8.850000
75%        13.800000
max        48.200000
Name: C6H6(GT), dtype: float64


### LSTM  

In [286]:
featuresCO

,PT08.S1(CO),T,RH,AH,Year_2005,Day,DayOfWeek,IsWeekend,Hour_sin,Hour_cos,Month_sin,Month_cos,Season_spring,Season_summer,PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3)
0,1143.0,12.0,61.6,0.8593,False,4,6,1,0.000000,1.000000,5.000000e-01,0.866025,False,False,825.0,986.0,1477.0,978.0
1,1186.0,15.8,48.0,0.8569,False,4,6,1,-0.500000,0.866025,5.000000e-01,0.866025,False,False,976.0,844.0,1570.0,1200.0
2,1192.0,16.8,44.0,0.8341,False,4,6,1,-0.707107,0.707107,5.000000e-01,0.866025,False,False,1007.0,827.0,1604.0,1223.0
3,1536.0,18.3,40.9,0.8493,False,4,6,1,-0.866025,0.500000,5.000000e-01,0.866025,False,False,1302.0,666.0,1922.0,1599.0
4,1592.0,20.8,36.7,0.8878,False,4,6,1,-0.965926,0.258819,5.000000e-01,0.866025,False,False,1429.0,624.0,2089.0,1644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7661,1128.0,10.2,57.9,0.7191,True,3,5,1,0.707107,0.707107,-2.449294e-16,1.000000,False,False,740.0,705.0,1113.0,1064.0
7662,1015.0,10.3,59.4,0.7459,True,3,5,1,0.866025,0.500000,-2.449294e-16,1.000000,False,False,608.0,873.0,1027.0,655.0
7663,979.0,9.8,68.9,0.8329,True,3,5,1,0.965926,0.258819,-2.449294e-16,1.000000,False,False,547.0,952.0,1039.0,470.0
7664,979.0,9.8,68.9,0.8329,True,3,5,1,0.965926,0.258819,-2.449294e-16,1.000000,False,False,547.0,952.0,1039.0,470.0


In [287]:
#LSTM pred for CO

# Features (all numeric columns)
X = featuresCO.values  # shape: (num_samples, num_features)

# Target
y = targetCO.values.reshape(-1, 1)  # make it 2D for scaler

# Scale features
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Function to create sequences
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

seq_length = 10  # number of past timesteps to use
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict
y_pred = model.predict(X_test)

# Inverse scale to original CO values
y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)

# Calculate RMSE
rmseLSTMCO = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
# Calculate R²
r2LSTMCO = r2_score(y_test_inv, y_pred_inv)
# Calculate MAE
maeLSTMCO = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"LSTM CO RMSE: {rmseLSTMCO:.4f}")
print(f"LSTM CO R2: {r2LSTMCO:.4f}")
print(f"LSTM CO MAE: {maeLSTMCO:.4f}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0210 - val_loss: 0.0141
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0117 - val_loss: 0.0128
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0109 - val_loss: 0.0113
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0108 - val_loss: 0.0131
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0104 - val_loss: 0.0101
Epoch 6/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0104 - val_loss: 0.0116
Epoch 7/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0102 - val_loss: 0.0101
Epoch 8/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0100 - val_loss: 0.0097
Epoch 9/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0099 - val_loss: 0.0116
Epoch 10/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0098 - val_loss: 0.0105
Epoch 11/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0097 - val_loss: 0.0091
Epoch 12/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [288]:
# LSTM pred for NOx

# Features (all numeric columns)
X = featuresNOx.values  # shape: (num_samples, num_features)

# Target
y = targetNOx.values.reshape(-1, 1)  # make it 2D for scaler

# Scale features
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

# Scale target
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Function to create sequences
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)
seq_length = 10  # number of past timesteps to use
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split  =0.1)

# Predict 
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

# Calculate RMSE
rmseLSTMNOx = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
# Calculate R²
r2LSTMNOx = r2_score(y_test_inv, y_pred_inv)   
# Calculate MAE
maeLSTMNOx = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"LSTM NOx RMSE: {rmseLSTMNOx:.4f}")
print(f"LSTM NOx R2: {r2LSTMNOx:.4f}")
print(f"LSTM NOx MAE: {maeLSTMNOx:.4f}")

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0119 - val_loss: 0.0190
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0081 - val_loss: 0.0192
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0076 - val_loss: 0.0184
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0073 - val_loss: 0.0245
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0070 - val_loss: 0.0238
Epoch 6/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0069 - val_loss: 0.0231
Epoch 7/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0067 - val_loss: 0.0231
Epoch 8/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0221
Epoch 9/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0264
Epoch 10/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0065 - val_loss: 0.0257
Epoch 11/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0063 - val_loss: 0.0290
Epoch 12/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.

In [289]:
# LSTM pred for NMHC

# Features (all numeric columns)
X = featuresNMHC.values  # shape: (num_samples, num_features)

# Target
y = targetNMHC.values.reshape(-1, 1)  # make it 2D for scaler

# Scale features
scaler_X = MinMaxScaler()
X = scaler_X.fit_transform(X)

# Scale target
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y)

# Function to create sequences
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)
seq_length = 10  # number of past timesteps to use
X_seq, y_seq = create_sequences(X, y, seq_length)

# Split into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)
# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

# Calculate RMSE
rmseLSTMNMHC = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
# Calculate R²
r2LSTMNMHC = r2_score(y_test_inv, y_pred_inv)   
# Calculate MAE
maeLSTMNMHC = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"LSTM NMHC RMSE: {rmseLSTMNMHC:.4f}")
print(f"LSTM NMHC R2: {r2LSTMNMHC:.4f}")
print(f"LSTM NMHC MAE: {maeLSTMNMHC:.4f}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0061 - val_loss: 0.0106
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - val_loss: 0.0109
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0081
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0097
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0042 - val_loss: 0.0115
Epoch 6/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040 - val_loss: 0.0095
Epoch 7/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0038 - val_loss: 0.0109
Epoch 8/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0037 - val_loss: 0.0115
Epoch 9/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0034 - val_loss: 0.0099
Epoch 10/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0035 - val_loss: 0.0108
Epoch 11/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0034 - val_loss: 0.0110
Epoch 12/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [290]:
# LSTM pred for NO2
# Features (all numeric columns)
X = featuresNO2.values  # shape: (num_samples, num_features)

# Target
y = targetNO2.values.reshape(-1, 1)  # make it 2D for scaler

# Scale features
scaler_X = MinMaxScaler()
X = scaler_X.fit_transform(X)

# Scale target
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y)

# Function to create sequences
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)
seq_length = 10  # number of past timesteps to use
X_seq, y_seq = create_sequences(X, y, seq_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

# Calculate RMSE
rmseLSTMNO2 = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
# Calculate R²
r2LSTMNO2 = r2_score(y_test_inv, y_pred_inv)   
# Calculate MAE
maeLSTMNO2 = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"LSTM NO2 RMSE: {rmseLSTMNO2:.4f}")
print(f"LSTM NO2 R2: {r2LSTMNO2:.4f}")
print(f"LSTM NO2 MAE: {maeLSTMNO2:.4f}")

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0081 - val_loss: 0.0171
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0073 - val_loss: 0.0171
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0068 - val_loss: 0.0155
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0064 - val_loss: 0.0150
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0064 - val_loss: 0.0151
Epoch 6/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0062 - val_loss: 0.0153
Epoch 7/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0061 - val_loss: 0.0116
Epoch 8/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0059 - val_loss: 0.0149
Epoch 9/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0059 - val_loss: 0.0127
Epoch 10/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0058 - val_loss: 0.0159
Epoch 11/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0058 - val_loss: 0.0112
Epoch 12/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.

In [291]:
# LSTM pred for C6H6
# Features (all numeric columns)
X = featuresC6H6.values  # shape: (num_samples, num_features)

# Target
y = targetC6H6.values.reshape(-1, 1)  # make it 2D for scaler

# Scale features
scaler_X = MinMaxScaler()
X = scaler_X.fit_transform(X)

# Scale target
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y)

# Function to create sequences
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)
seq_length = 10  # number of past timesteps to use
X_seq, y_seq = create_sequences(X, y, seq_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)
# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1) 

# Predict
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

# Calculate RMSE
rmseLSTMC6H6 = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
# Calculate R²
r2LSTMC6H6 = r2_score(y_test_inv, y_pred_inv)   
# Calculate MAE
maeLSTMC6H6 = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"LSTM C6H6 RMSE: {rmseLSTMC6H6:.4f}")
print(f"LSTM C6H6 R2: {r2LSTMC6H6:.4f}")
print(f"LSTM C6H6 MAE: {maeLSTMC6H6:.4f}")

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0167 - val_loss: 0.0110
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0138 - val_loss: 0.0132
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0129 - val_loss: 0.0114
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0126 - val_loss: 0.0112
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0124 - val_loss: 0.0111
Epoch 6/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0123 - val_loss: 0.0108
Epoch 7/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0122 - val_loss: 0.0115
Epoch 8/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119 - val_loss: 0.0114
Epoch 9/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119 - val_loss: 0.0096
Epoch 10/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0116 - val_loss: 0.0103
Epoch 11/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0114 - val_loss: 0.0098
Epoch 12/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.

### Compare Results

In [292]:
# Collect metrics for each chemical and method

# Baseline
baseline_metrics = baseline_results_df[['RMSE', 'R2', 'MAE']].copy()
baseline_metrics.columns = ['Baseline_RMSE', 'Baseline_R2', 'Baseline_MAE']

# Random Forest
rf_metrics = pd.DataFrame({
    'RandomForest_RMSE': [
        mseCO**0.5, mseC6H6**0.5, mseNMHC**0.5, mseNOx**0.5, mseNO2**0.5
    ],
    'RandomForest_R2': [
        r2CO, r2C6H6, r2NMHC, r2NOx, r2NO2
    ],
    'RandomForest_MAE': [
        maeCO, maeC6H6, maeNMHC, maeNOx, maeNO2
    ]
}, index=baseline_metrics.index)

# LSTM
lstm_metrics = pd.DataFrame({
    'LSTM_RMSE': [
        rmseLSTMCO, rmseLSTMC6H6, rmseLSTMNMHC, rmseLSTMNOx, rmseLSTMNO2
    ],
    'LSTM_R2': [
        r2LSTMCO, r2LSTMC6H6, r2LSTMNMHC, r2LSTMNOx, r2LSTMNO2
    ],
    'LSTM_MAE': [
        maeLSTMCO, maeLSTMC6H6, maeLSTMNMHC, maeLSTMNOx, maeLSTMNO2
    ]
}, index=baseline_metrics.index)

# Combine all metrics
results_df = pd.concat([baseline_metrics, rf_metrics, lstm_metrics], axis=1)
results_df

,Baseline_RMSE,Baseline_R2,Baseline_MAE,RandomForest_RMSE,RandomForest_R2,RandomForest_MAE,LSTM_RMSE,LSTM_R2,LSTM_MAE
CO(GT),1.141277,0.243215,0.684330,0.804571,6.632457e-01,0.538787,1.287906,1.379575e-01,0.931257
C6H6(GT),33.390092,0.313139,3.736177,0.029435,9.999762e-01,0.012937,6.073926,-1.376585e-02,4.559173
NMHC(GT),3126.714984,0.324911,12.280400,63.150930,-4.936954e+30,41.548760,128.514971,-2.044594e+31,111.468115
NOx(GT),17431.584722,0.455097,78.551155,142.815136,5.778612e-01,96.461562,286.766760,-7.006490e-01,208.082472
NO2(GT),1030.288784,0.446477,21.128452,40.870209,3.507364e-01,29.553529,77.727528,-1.347183e+00,63.834798
